In [1]:
import pandas as pd


In [2]:
#Lectura del archivo con la informacion a trabajar obtenida de Kaggle
df = pd.read_csv("../data/processed/df_final.csv")
df

,Venue,Result,Opponent,xG,xGA,Referee,Team
0,Away,0,Chelsea,1.3,1.4,Anthony Taylor,Liverpool
1,Home,1,Bournemouth,3.0,1.3,Thomas Bramall,Liverpool
2,Away,1,Newcastle Utd,0.9,2.0,John Brooks,Liverpool
3,Home,1,Aston Villa,2.5,0.7,Simon Hooper,Liverpool
4,Away,1,Wolves,2.5,0.6,Michael Oliver,Liverpool
...,...,...,...,...,...,...,...
507,Away,2,Crystal Palace,0.8,1.1,Tony Harrington,SheffieldUnited
508,Home,2,Aston Villa,1.4,2.2,Paul Tierney,SheffieldUnited
509,Away,1,Luton Town,1.7,1.8,Chris Kavanagh,SheffieldUnited
510,Home,2,Brighton,0.6,2.9,Stuart Attwell,SheffieldUnited


In [3]:
X = df.drop(columns=["Result"])
y = df['Result']

In [5]:
X.head()

,Venue,Opponent,xG,xGA,Referee,Team
0,Away,Chelsea,1.3,1.4,Anthony Taylor,Liverpool
1,Home,Bournemouth,3.0,1.3,Thomas Bramall,Liverpool
2,Away,Newcastle Utd,0.9,2.0,John Brooks,Liverpool
3,Home,Aston Villa,2.5,0.7,Simon Hooper,Liverpool
4,Away,Wolves,2.5,0.6,Michael Oliver,Liverpool


In [7]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
X= encoder.fit_transform(X)

In [10]:
# Guardar label en pickle
import pickle 

with open('x_encode.pkl', 'wb') as f:
    pickle.dump(encoder, f)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
import dagshub
import mlflow

dagshub.init("PROYECTO-CD-Premiere-league", "SantiagoAguirre2002", mlflow=True)

mlflow.set_tracking_uri('https://dagshub.com/IlseFlores/PROYECTO-CD-Premiere-league.mlflow')

Initialized MLflow to track repo "SantiagoAguirre2002/PROYECTO-CD-Premiere-league"

Repository SantiagoAguirre2002/PROYECTO-CD-Premiere-league initialized!

In [13]:
 #pip install mlflow
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

name_model = "LogisticRegression"
developer = "Angel De la mora, Regina Flores, Santiago Aguirre"

parameters = {
    "penalty": "l2",
    "C": 1.0,
    "solver": "lbfgs",
    "multi_class": "multinomial", 
    "max_iter": 100,
    "random_state": 42
}

with mlflow.start_run(run_name=name_model):
    
    mlflow.log_param("model", name_model)
    mlflow.log_param("developer", developer)
    
    # Registrar parámetros del modelo
    mlflow.log_params(parameters)
    
    # Entrenar un clasificador de regresión logística
    logistic_regression_clf = LogisticRegression(**parameters)
    logistic_regression_clf.fit(X_train, y_train)
    
    # Realizar predicciones
    predictions = logistic_regression_clf.predict(X_test)
    predictions_proba = logistic_regression_clf.predict_proba(X_test)

    # Calcular métricas
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    roc_auc = roc_auc_score(y_test, predictions_proba, average='weighted', multi_class='ovr')

    # Registrar métricas
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    
    # Registrar el modelo
    mlflow.sklearn.log_model(logistic_regression_clf, "logistic_regression_classifier")


C:\Users\tatob\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Experimento1

In [ ]:
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

name_model = "LogisticRegression"
developer = "Angel De la mora, Regina Flores, Santiago Aguirre"

parameters = {
    "penalty": "l2",
    "C": 1.0,
    "solver": "lbfgs",
    "multi_class": "multinomial", 
    "max_iter": 100,
    "random_state": 42
}

with mlflow.start_run(run_name=name_model):
    
    mlflow.log_param("model", name_model)
    mlflow.log_param("developer", developer)
    
    # Registrar parámetros del modelo
    mlflow.log_params(parameters)
    
    # Entrenar un clasificador de regresión logística
    logistic_regression_clf = LogisticRegression(**parameters)
    logistic_regression_clf.fit(X_train, y_train)
    
    # Realizar predicciones
    predictions = logistic_regression_clf.predict(X_test)
    predictions_proba = logistic_regression_clf.predict_proba(X_test)

    # Calcular métricas
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    roc_auc = roc_auc_score(y_test, predictions_proba, average='weighted', multi_class='ovr')

    # Registrar métricas
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    
    # Registrar el modelo
    mlflow.sklearn.log_model(logistic_regression_clf, "logistic_regression_classifier")


## Experimento 2

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

param_grid = {
    'hidden_layer_sizes': [(100,), (50, 50), (50, 100, 50)],
    'activation': ['logistic', 'relu'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'adaptive'],
}

mlp_clf = MLPClassifier(random_state=42)

grid_search = GridSearchCV(estimator=mlp_clf, param_grid=param_grid, cv=5)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Mejores parámetros:", best_params)


In [ ]:
import mlflow
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

name_model = "MLPClassifier"
developer = "Angel De la mora, Regina Flores, Santiago Aguirre"

# Mejores parámetros encontrados
best_params = {
    "hidden_layer_sizes": (50, 50),
    "activation": "logistic",
    "solver": "adam",
    "alpha": 0.0001,
    "learning_rate": "constant",
    "max_iter": 1000,  # Aumentamos el número de iteraciones
    "random_state": 42
}

with mlflow.start_run(run_name=name_model):
    
    mlflow.log_param("model", name_model)
    mlflow.log_param("developer", developer)
    
    # Registrar parámetros del modelo
    mlflow.log_params(best_params)
    
    # Entrenar un clasificador de red neuronal
    mlp_clf = MLPClassifier(**best_params)
    mlp_clf.fit(X_train, y_train)
    
    # Realizar predicciones
    predictions = mlp_clf.predict(X_test)
    predictions_proba = mlp_clf.predict_proba(X_test)

    # Calcular métricas
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    roc_auc = roc_auc_score(y_test, predictions_proba, average='weighted')

    # Registrar métricas
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    
    # Registrar el modelo
    mlflow.sklearn.log_model(mlp_clf, "mlp_classifier")


### Experimento 3


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 200, 300],  
    'max_depth': [None, 10, 20, 30],  
    'min_samples_split': [2, 5, 10],   
    'min_samples_leaf': [1, 2, 4],  
    
    'max_features': ['auto', 'sqrt'],  
}

rf_clf = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, cv=5)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Mejores parámetros:", best_params)

In [ ]:
import mlflow
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

name_model = "RandomForestClassifier"
developer = "Angel De la mora, Regina Flores, Santiago Aguirre"

# Mejores parámetros encontrados
best_params = {
    'max_depth': None,
    'max_features': 'sqrt',
    'min_samples_leaf': 1,
    'min_samples_split': 5,
    'n_estimators': 200,
    'random_state': 42
}

with mlflow.start_run(run_name=name_model):
    
    mlflow.log_param("model", name_model)
    mlflow.log_param("developer", developer)
    
    # Registrar parámetros del modelo
    mlflow.log_params(best_params)
    
    # Entrenar un clasificador de Random Forest
    rf_clf = RandomForestClassifier(**best_params)
    rf_clf.fit(X_train, y_train)
    
    # Realizar predicciones
    predictions = rf_clf.predict(X_test)
    predictions_proba = rf_clf.predict_proba(X_test)

    # Calcular métricas
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    roc_auc = roc_auc_score(y_test, predictions_proba, average='weighted', multi_class='ovr')

    # Registrar métricas
    mlflow.log_metric("roc_auc", roc_auc)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    
    # Registrar el modelo
    mlflow.sklearn.log_model(rf_clf, "random_forest_classifier")